<h1>Toronto Clustering</h1>

In [2]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
import matplotlib.cm as cm
import matplotlib.colors as colors
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


/Users/novfrost/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [25]:
df = pd.read_csv("toronto.csv")
df.head()

Postal Code           Borough               Neighborhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront

In [4]:
df.shape

(180, 3)

In [31]:
# Drop rows that does not have a Borough
condition = df['Borough'] == "Not assigned"
df.drop(df[condition].index, axis=0, inplace=True)
df.head(10)

Postal Code           Borough                                 Neighborhood
2          M3A        North York                                    Parkwoods
3          M4A        North York                             Victoria Village
4          M5A  Downtown Toronto                    Regent Park, Harbourfront
5          M6A        North York             Lawrence Manor, Lawrence Heights
6          M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government
8          M9A         Etobicoke      Islington Avenue, Humber Valley Village
9          M1B       Scarborough                               Malvern, Rouge
11         M3B        North York                                    Don Mills
12         M4B         East York              Parkview Hill, Woodbine Gardens
13         M5B  Downtown Toronto                     Garden District, Ryerson

In [28]:
df.shape

(103, 3)

In [36]:
df_zip = pd.read_csv("Geospatial_Coordinates.csv")
df_zip.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [45]:
df_coor = pd.merge(df, df_zip, on='Postal Code')
df_coor

Postal Code           Borough  \
0           M3A        North York   
1           M4A        North York   
2           M5A  Downtown Toronto   
3           M6A        North York   
4           M7A  Downtown Toronto   
..          ...               ...   
98          M8X         Etobicoke   
99          M4Y  Downtown Toronto   
100         M7Y      East Toronto   
101         M8Y         Etobicoke   
102         M8Z         Etobicoke   

                                          Neighborhood   Latitude  Longitude  
0                                            Parkwoods  43.753259 -79.329656  
1                                     Victoria Village  43.725882 -79.315572  
2                            Regent Park, Harbourfront  43.654260 -79.360636  
3                     Lawrence Manor, Lawrence Heights  43.718518 -79.464763  
4          Queen's Park, Ontario Provincial Government  43.662301 -79.389494  
..                                                 ...        ...        ...  
98       The Kingsway, Montgomery Road, Old Mill North  43.653654 -79.506944  
99                                Church and Wellesley  43.665860 -79.383160  
100  Business reply mail Processing Centre, South C...  43.662744 -79.321558  
101  Old Mill South, King's Mill Park, Sunnylea, Hu...  43.636258 -79.498509  
102  Mimico NW, The Queensway West, South of Bloor,...  43.628841 -79.520999  

[103 rows x 5 columns]

<h1>Map of Toronto</h1>

In [46]:
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [48]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_coor['Latitude'], df_coor['Longitude'], df_coor['Borough'], df_coor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<h1>Toronto Clustering</h1>

In [51]:
toronto_data = df_coor[df_coor['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

Postal Code           Borough                                 Neighborhood  \
0         M5A  Downtown Toronto                    Regent Park, Harbourfront   
1         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
2         M5B  Downtown Toronto                     Garden District, Ryerson   
3         M5C  Downtown Toronto                               St. James Town   
4         M4E      East Toronto                                  The Beaches   

    Latitude  Longitude  
0  43.654260 -79.360636  
1  43.662301 -79.389494  
2  43.657162 -79.378937  
3  43.651494 -79.375418  
4  43.676357 -79.293031

In [55]:
# create map of New York using latitude and longitude values
map_toronto_2 = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_2)  
    
map_toronto_2

In [78]:
CLIENT_ID = 'KXTFRJ42N1AM00ZP22TQWVRXN3K3XO4ZG40GGQ12WVBAXNL4' # your Foursquare ID
CLIENT_SECRET = 'MWH2TJ121VF3RML1WLF0RJDQGLABXRFONKTTJ23SUHAFYYDW' # your Foursquare Secret
VERSION = '2020622' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KXTFRJ42N1AM00ZP22TQWVRXN3K3XO4ZG40GGQ12WVBAXNL4
CLIENT_SECRET:MWH2TJ121VF3RML1WLF0RJDQGLABXRFONKTTJ23SUHAFYYDW


In [79]:
toronto_data.loc[0, 'Neighborhood']

'Regent Park, Harbourfront'

In [80]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


In [83]:
search_query = 'Regent Park, Harbourfront'
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id=KXTFRJ42N1AM00ZP22TQWVRXN3K3XO4ZG40GGQ12WVBAXNL4&client_secret=MWH2TJ121VF3RML1WLF0RJDQGLABXRFONKTTJ23SUHAFYYDW&ll=43.65096410900003,-79.35304116399999&v=20200622&radius=500&limit=100'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=KXTFRJ42N1AM00ZP22TQWVRXN3K3XO4ZG40GGQ12WVBAXNL4&client_secret=MWH2TJ121VF3RML1WLF0RJDQGLABXRFONKTTJ23SUHAFYYDW&ll=43.65096410900003,-79.35304116399999&v=20200622&radius=500&limit=100'

In [84]:
results = requests.get(url).json()
results
results['response']['groups'][0]['items']

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '57e0745a498ea809dbf75f68',
   'name': 'Souk Tabule',
   'location': {'address': '494 Front St E',
    'crossStreet': 'at Bayview',
    'lat': 43.65375556880743,
    'lng': -79.35439006096168,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.65375556880743,
      'lng': -79.35439006096168}],
    'distance': 329,
    'cc': 'CA',
    'neighborhood': 'Canary District',
    'city': 'Toronto',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['494 Front St E (at Bayview)',
     'Toronto ON',
     'Canada']},
   'categories': [{'id': '4bf58dd8d48988d1c0941735',
     'name': 'Mediterranean Restaurant',
     'pluralName': 'Mediterranean Restaurants',
     'shortName': 'Mediterranean',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mediterranean_',
      'suffix': '.png'},
     'p

In [85]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [86]:
venues=results['response']['groups'][0]['items']

In [87]:
nearby_venues = pd.json_normalize(venues) # flatten JSON

In [89]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

In [90]:
#filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

In [91]:
#clean columns
nearby_venues.columns=[col.split('.')[-1] for col in nearby_venues.columns]
nearby_venues.head()

name                categories        lat  \
0                           Souk Tabule  Mediterranean Restaurant  43.653756   
1  Young Centre for the Performing Arts     Performing Arts Venue  43.650825   
2            Cluny Bistro & Boulangerie         French Restaurant  43.650565   
3                           BATLgrounds        Athletics & Sports  43.647088   
4                   SOMA chocolatemaker            Chocolate Shop  43.650622   

         lng  
0 -79.354390  
1 -79.357593  
2 -79.357843  
3 -79.351306  
4 -79.358127

In [92]:
# How many venus were returnes by Foursquare?
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

27 venues were returned by Foursquare.


<h1>Explore Neighborhood in Toronto</h1>

In [112]:
CLIENT_ID = 'KXTFRJ42N1AM00ZP22TQWVRXN3K3XO4ZG40GGQ12WVBAXNL4' # your Foursquare ID
CLIENT_SECRET = 'MWH2TJ121VF3RML1WLF0RJDQGLABXRFONKTTJ23SUHAFYYDW' # your Foursquare Secret
VERSION = '20140609' # Foursquare API version
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [113]:
toronto_data.tail(3)

Postal Code           Borough  \
36         M5X  Downtown Toronto   
37         M4Y  Downtown Toronto   
38         M7Y      East Toronto   

                                         Neighborhood   Latitude  Longitude  
36             First Canadian Place, Underground city  43.648429 -79.382280  
37                               Church and Wellesley  43.665860 -79.383160  
38  Business reply mail Processing Centre, South C...  43.662744 -79.321558

In [114]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                              latitudes=toronto_data['Latitude'],
                              longitudes=toronto_data['Longitude'])

Regent Park, Harbourfront
https://api.foursquare.com/v2/venues/explore?&client_id=KXTFRJ42N1AM00ZP22TQWVRXN3K3XO4ZG40GGQ12WVBAXNL4&client_secret=MWH2TJ121VF3RML1WLF0RJDQGLABXRFONKTTJ23SUHAFYYDW&v=20140609&ll=43.6542599,-79.3606359&radius=500&limit=100
Queen's Park, Ontario Provincial Government
https://api.foursquare.com/v2/venues/explore?&client_id=KXTFRJ42N1AM00ZP22TQWVRXN3K3XO4ZG40GGQ12WVBAXNL4&client_secret=MWH2TJ121VF3RML1WLF0RJDQGLABXRFONKTTJ23SUHAFYYDW&v=20140609&ll=43.6623015,-79.3894938&radius=500&limit=100
Garden District, Ryerson
https://api.foursquare.com/v2/venues/explore?&client_id=KXTFRJ42N1AM00ZP22TQWVRXN3K3XO4ZG40GGQ12WVBAXNL4&client_secret=MWH2TJ121VF3RML1WLF0RJDQGLABXRFONKTTJ23SUHAFYYDW&v=20140609&ll=43.6571618,-79.37893709999999&radius=500&limit=100
St. James Town
https://api.foursquare.com/v2/venues/explore?&client_id=KXTFRJ42N1AM00ZP22TQWVRXN3K3XO4ZG40GGQ12WVBAXNL4&client_secret=MWH2TJ121VF3RML1WLF0RJDQGLABXRFONKTTJ23SUHAFYYDW&v=20140609&ll=43.6514939,-79.3754179&

CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
https://api.foursquare.com/v2/venues/explore?&client_id=KXTFRJ42N1AM00ZP22TQWVRXN3K3XO4ZG40GGQ12WVBAXNL4&client_secret=MWH2TJ121VF3RML1WLF0RJDQGLABXRFONKTTJ23SUHAFYYDW&v=20140609&ll=43.6289467,-79.3944199&radius=500&limit=100
Rosedale
https://api.foursquare.com/v2/venues/explore?&client_id=KXTFRJ42N1AM00ZP22TQWVRXN3K3XO4ZG40GGQ12WVBAXNL4&client_secret=MWH2TJ121VF3RML1WLF0RJDQGLABXRFONKTTJ23SUHAFYYDW&v=20140609&ll=43.6795626,-79.37752940000001&radius=500&limit=100
Stn A PO Boxes
https://api.foursquare.com/v2/venues/explore?&client_id=KXTFRJ42N1AM00ZP22TQWVRXN3K3XO4ZG40GGQ12WVBAXNL4&client_secret=MWH2TJ121VF3RML1WLF0RJDQGLABXRFONKTTJ23SUHAFYYDW&v=20140609&ll=43.6464352,-79.37484599999999&radius=500&limit=100
St. James Town, Cabbagetown
https://api.foursquare.com/v2/venues/explore?&client_id=KXTFRJ42N1AM00ZP22TQWVRXN3K3XO4ZG40GGQ12WVBAXNL4&client_secret=MWH2TJ121VF3RML1WLF0RJDQGLABXR

In [115]:
# Resulting dataframe
print(toronto_venues.shape)
toronto_venues.head()

(1621, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  Regent Park, Harbourfront               43.65426              -79.360636   
1  Regent Park, Harbourfront               43.65426              -79.360636   
2  Regent Park, Harbourfront               43.65426              -79.360636   
3  Regent Park, Harbourfront               43.65426              -79.360636   
4  Regent Park, Harbourfront               43.65426              -79.360636   

                      Venue  Venue Latitude  Venue Longitude  \
0          Roselle Desserts       43.653447       -79.362017   
1             Tandem Coffee       43.653559       -79.361809   
2    Cooper Koo Family YMCA       43.653249       -79.358008   
3       Body Blitz Spa East       43.654735       -79.359874   
4  Dominion Pub and Kitchen       43.656919       -79.358967   

        Venue Category  
0               Bakery  
1          Coffee Shop  
2  Distribution Center  
3                  Spa  
4                  Pub

In [117]:
# Venus for each N
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                            56   
Brockton, Parkdale Village, Exhibition Place                           24   
Business reply mail Processing Centre, South Ce...                     16   
CN Tower, King and Spadina, Railway Lands, Harb...                     16   
Central Bay Street                                                     64   
Christie                                                               17   
Church and Wellesley                                                   75   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             33   
Davisville North                                                        9   
Dufferin, Dovercourt Village                                           16   
First Canadian Place, Underground city                                100   
Forest Hill North & West, Forest Hill Road Park                         6   
Garden District, Ryerson                                              100   
Harbourfront East, Union Station, Toronto Islands                     100   
High Park, The Junction South                                          25   
India Bazaar, The Beaches West                                         24   
Kensington Market, Chinatown, Grange Park                              62   
Lawrence Park                                                           4   
Little Portugal, Trinity                                               44   
Moore Park, Summerhill East                                             3   
North Toronto West, Lawrence Park                                      20   
Parkdale, Roncesvalles                                                 15   
Queen's Park, Ontario Provincial Government                            34   
Regent Park, Harbourfront                                              44   
Richmond, Adelaide, King                                               92   
Rosedale                                                                4   
Roselawn                                                                1   
Runnymede, Swansea                                                     38   
St. James Town                                                         78   
St. James Town, Cabbagetown                                            46   
Stn A PO Boxes                                                         96   
Studio District                                                        40   
Summerhill West, Rathnelly, South Hill, Forest ...                     16   
The Annex, North Midtown, Yorkville                                    21   
The Beaches                                                             5   
The Danforth West, Riverdale                                           43   
Toronto Dominion Centre, Design Exchange                              100   
University of Toronto, Harbord                                         34   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Berczy Park                                                             56   
Brockton, Parkdale Village, Exhibition Place                            24   
Business reply mail Processing Centre, South Ce...                      16   
CN Tower, King and Spadina, Railway Lands, Harb...                      16   
Central Bay Street                                                      64   
Christie                                                                17   
Church and Wellesley                                                    75   
Commerce Court, Victoria Hotel                                         100   
Davisville                                                              33   
Davisville North                    

In [118]:
# Venus unique categories
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 238 uniques categories.


In [120]:
# One hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot

Afghan Restaurant  Airport  Airport Food Court  Airport Gate  \
0                     0        0                   0             0   
1                     0        0                   0             0   
2                     0        0                   0             0   
3                     0        0                   0             0   
4                     0        0                   0             0   
...                 ...      ...                 ...           ...   
1616                  0        0                   0             0   
1617                  0        0                   0             0   
1618                  0        0                   0             0   
1619                  0        0                   0             0   
1620                  0        0                   0             0   

      Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0                  0                0                 0                    0   
1                  0                0                 0                    0   
2                  0                0                 0                    0   
3                  0                0                 0                    0   
4                  0                0                 0                    0   
...              ...              ...               ...                  ...   
1616               0                0                 0                    0   
1617               0                0                 0                    0   
1618               0                0                 0                    0   
1619               0                0                 0                    0   
1620               0                0                 0                    0   

      Antique Shop  Aquarium  ...  Toy / Game Store  Trail  Train Station  \
0                0         0  ...                 0      0              0   
1                0         0  ...                 0      0              0   
2                0         0  ...                 0      0              0   
3                0         0  ...                 0      0              0   
4                0         0  ...                 0      0              0   
...            ...       ...  ...               ...    ...            ...   
1616             0         0  ...                 0      0              0   
1617             0         0  ...                 0      0              0   
1618             0         0  ...                 0      0              0   
1619             0         0  ...                 0      0              0   
1620             0         0  ...                 0      0              0   

      Vegetarian / Vegan Restaurant  Video Game Store  Vietnamese Restaurant  \
0                                 0                 0                      0   
1                                 0                 0                      0   
2                                 0                 0                      0   
3                                 0                 0                      0   
4                                 0                 0                      0   
...                             ...               ...                    ...   
1616                              0                 0                      0   
1617                              0                 0                      0   
1618                              0                 0                      0   
1619                              0                 0                      0   
1620                              0                 0                      0   

      Wine Bar  Wine Shop  Women's Store  Yoga Studio  
0            0          0              0            0  
1            0          0              0            0  
2            0          0              0            0  
3            0          0              0            0  
4            0          0             

In [129]:
toronto_onehot.drop('Neighborhood', axis=1, inplace=True)

In [130]:
# add neighborhood column back to dataframe
neighborhood=toronto_venues['Neighborhood'].values
toronto_onehot.insert(0, 'Neighborhood', neighborhood, allow_duplicates=False)

# move neighborhood column to the first column
fixed_columns=list(toronto_onehot.columns)
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Neighborhood  Women's Store  Yoga Studio  Afghan Restaurant  \
0  Regent Park, Harbourfront              0            0                  0   
1  Regent Park, Harbourfront              0            0                  0   
2  Regent Park, Harbourfront              0            0                  0   
3  Regent Park, Harbourfront              0            0                  0   
4  Regent Park, Harbourfront              0            0                  0   

   Airport  Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0        0                   0             0               0                0   
1        0                   0             0               0                0   
2        0                   0             0               0                0   
3        0                   0             0               0                0   
4        0                   0             0               0                0   

   Airport Terminal  ...  Theater  Theme Restaurant  Toy / Game Store  Trail  \
0                 0  ...        0                 0                 0      0   
1                 0  ...        0                 0                 0      0   
2                 0  ...        0                 0                 0      0   
3                 0  ...        0                 0                 0      0   
4                 0  ...        0                 0                 0      0   

   Train Station  Vegetarian / Vegan Restaurant  Video Game Store  \
0              0                              0                 0   
1              0                              0                 0   
2              0                              0                 0   
3              0                              0                 0   
4              0                              0                 0   

   Vietnamese Restaurant  Wine Bar  Wine Shop  
0                      0         0          0  
1                      0         0          0  
2                      0         0          0  
3                      0         0          0  
4                      0         0          0  

[5 rows x 238 columns]

In [131]:
toronto_onehot.shape

(1621, 238)

In [132]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Neighborhood  Women's Store  \
0                                         Berczy Park        0.00000   
1        Brockton, Parkdale Village, Exhibition Place        0.00000   
2   Business reply mail Processing Centre, South C...        0.00000   
3   CN Tower, King and Spadina, Railway Lands, Har...        0.00000   
4                                  Central Bay Street        0.00000   
5                                            Christie        0.00000   
6                                Church and Wellesley        0.00000   
7                      Commerce Court, Victoria Hotel        0.00000   
8                                          Davisville        0.00000   
9                                    Davisville North        0.00000   
10                       Dufferin, Dovercourt Village        0.00000   
11             First Canadian Place, Underground city        0.00000   
12    Forest Hill North & West, Forest Hill Road Park        0.00000   
13                           Garden District, Ryerson        0.00000   
14  Harbourfront East, Union Station, Toronto Islands        0.00000   
15                      High Park, The Junction South        0.00000   
16                     India Bazaar, The Beaches West        0.00000   
17          Kensington Market, Chinatown, Grange Park        0.00000   
18                                      Lawrence Park        0.00000   
19                           Little Portugal, Trinity        0.00000   
20                        Moore Park, Summerhill East        0.00000   
21                  North Toronto West, Lawrence Park        0.00000   
22                             Parkdale, Roncesvalles        0.00000   
23        Queen's Park, Ontario Provincial Government        0.00000   
24                          Regent Park, Harbourfront        0.00000   
25                           Richmond, Adelaide, King        0.01087   
26                                           Rosedale        0.00000   
27                                           Roselawn        0.00000   
28                                 Runnymede, Swansea        0.00000   
29                                     St. James Town        0.00000   
30                        St. James Town, Cabbagetown        0.00000   
31                                     Stn A PO Boxes        0.00000   
32                                    Studio District        0.00000   
33  Summerhill West, Rathnelly, South Hill, Forest...        0.00000   
34                The Annex, North Midtown, Yorkville        0.00000   
35                                        The Beaches        0.00000   
36                       The Danforth West, Riverdale        0.00000   
37           Toronto Dominion Centre, Design Exchange        0.00000   
38                     University of Toronto, Harbord        0.00000   

    Yoga Studio  Afghan Restaurant  Airport  Airport Food Court  Airport Gate  \
0      0.000000           0.000000   0.0000              0.0000        0.0000   
1      0.041667           0.000000   0.0000              0.0000        0.0000   
2      0.000000           0.000000   0.0000              0.0000        0.0000   
3      0.000000           0.000000   0.0625              0.0625        0.0625   
4      0.015625           0.000000   0.0000              0.0000        0.0000   
5      0.000000           0.000000   0.0000              0.0000        0.0000   
6      0.026667           0.013333   0.0000              0.0000        0.0000   
7      0.000000           0.000000   0.0000              0.0000        0.0000   
8      0.000000           0.000000   0.0000              0.0000        0.0000   
9      0.000000           0.000000   0.0000              0.0000        0.0000   
10     0.000000           0.000000   0.0000              0.0000        0.0000   
11     0.000000           0.000000   0.0000              0.0000        0.0000   
12     0.000000           0.000000   0.0000              0.0000        0.0000   
13     0.000000           

In [134]:
toronto_grouped.shape

(39, 238)

In [135]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2              Bakery  0.04
3  Seafood Restaurant  0.04
4                Café  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.12
1  Breakfast Spot  0.08
2          Bakery  0.08
3     Coffee Shop  0.08
4    Climbing Gym  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0        Farmers Market  0.06
1                Garden  0.06
2                  Park  0.06
3  Gym / Fitness Center  0.06
4           Pizza Place  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.19
1  Airport Terminal  0.12
2    Airport Lounge  0.12
3   Harbor / Marina  0.06
4     Boat or Ferry  0.06


----Central Bay Street----
                 venue  freq
0      

In [136]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [137]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                        Berczy Park           Coffee Shop   
1       Brockton, Parkdale Village, Exhibition Place                  Café   
2  Business reply mail Processing Centre, South C...         Auto Workshop   
3  CN Tower, King and Spadina, Railway Lands, Har...       Airport Service   
4                                 Central Bay Street           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0          Cocktail Bar           Cheese Shop                  Café   
1        Breakfast Spot           Coffee Shop                Bakery   
2  Gym / Fitness Center  Fast Food Restaurant        Farmers Market   
3      Airport Terminal        Airport Lounge                 Plane   
4        Sandwich Place    Italian Restaurant                  Café   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0               Beer Bar    Seafood Restaurant            Restaurant   
1  Performing Arts Venue             Nightclub    Italian Restaurant   
2                    Spa            Restaurant           Pizza Place   
3               Boutique   Rental Car Location      Sculpture Garden   
4    Japanese Restaurant           Salad Place       Bubble Tea Shop   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0                Bakery              Creperie                   Park  
1            Restaurant          Intersection          Burrito Place  
2               Brewery         Burrito Place             Comic Shop  
3       Harbor / Marina         Boat or Ferry           Airport Gate  
4       Thai Restaurant          Burger Joint       Department Store

<h1>Cluster</h1>

In [150]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [151]:
neighborhoods_venues_sorted.drop('Cluster Labels', axis=1, inplace=True)

In [152]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

Postal Code           Borough                                 Neighborhood  \
0         M5A  Downtown Toronto                    Regent Park, Harbourfront   
1         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
2         M5B  Downtown Toronto                     Garden District, Ryerson   
3         M5C  Downtown Toronto                               St. James Town   
4         M4E      East Toronto                                  The Beaches   

    Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
0  43.654260 -79.360636               2           Coffee Shop   
1  43.662301 -79.389494               2           Coffee Shop   
2  43.657162 -79.378937               2        Clothing Store   
3  43.651494 -79.375418               2                  Café   
4  43.676357 -79.293031               0      Asian Restaurant   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                  Park                Bakery                   Pub   
1                 Diner      Sushi Restaurant           Music Venue   
2           Coffee Shop    Italian Restaurant        Cosmetics Shop   
3           Coffee Shop          Cocktail Bar            Restaurant   
4                   Pub                 Trail     Health Food Store   

  5th Most Common Venue      6th Most Common Venue 7th Most Common Venue  \
0                  Café                 Restaurant        Breakfast Spot   
1    Mexican Restaurant             Discount Store        Sandwich Place   
2                  Café  Middle Eastern Restaurant       Bubble Tea Shop   
3   American Restaurant                  Gastropub        Clothing Store   
4          Dessert Shop               Cupcake Shop          Dance Studio   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0               Theater           Event Space             Beer Store  
1    Italian Restaurant      Sculpture Garden               Beer Bar  
2   Japanese Restaurant             Bookstore                Theater  
3                   Gym      Department Store                  Hotel  
4         Deli / Bodega      Department Store              Wine Shop

In [153]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [154]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
4  East Toronto               0      Asian Restaurant                   Pub   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
4                 Trail     Health Food Store          Dessert Shop   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
4          Cupcake Shop          Dance Studio         Deli / Bodega   

  9th Most Common Venue 10th Most Common Venue  
4      Department Store              Wine Shop